## Подготовка
Установка необходимых зависимостей:

In [ ]:
!pip install -r requirements.txt

## Постановка задачи
Пусть дан конечный ориентированный помеченный граф $G = (V, E, M)$, в котором каждому пути $l \in E^*$, состоящему из рёбер с метками $m_1, ..., m_n \in M$, сопоставляется слово $\omega: \omega(l) = m_1...m_n$ (конкатенация меток).

Пусть также дано регулярное выражение $R$, порождающее язык $L^R$.

В рамках данного эксперимента будет рассмотренно три варианта постановки задачи о достижимости в графе $G$ с регулярными ограничениями, задаваемыми $R$.

### 1. Достижимость между всеми парами вершин
Требуется найти все пары вершин из $G$, между которомы существует путь в графе, сопоставимый слову из языка $L^R$. То есть:

$\{(v_i, v_j)\in V^2: \exists l \in E^*: start(l) = v_i, final(l) = v_j, \omega(l)\in L^R\}$.

Для дальнейших постановок задач введём обозначения:
$V_s$ - множество стартовых вершин, $V_f$ - множество финальных вершин графа $G$.

### 2. Достижимость для всего множества заданных вершин
Требуется найти все финальные вершины такие, что до них существуют пути из заданных стартовых вершин, сопоставимые словам языка $L^R$. То есть:

$\{v_j\in V_f: \exists l\in E^*: start(l)\in V_s, final(l) = v_j, \omega(l)\in L^R\}$.

### 3. Достижимость для каждой вершины из заданного множества стартовых вершин
Требуется найти множество пар стартовых и финальных вершин для путей, сопоставимых слову из языка $L^R$. То есть:

$\{(v_i, v_j)\in V_s\times V_f:\exists l\in E^*: start(l) = v_i, final(l) = v_j,\omega(l)\in L^R\}$.

## Описание исследуемых решений
### 1. Достижимость между всеми парами вершин
По графу и регулярному выражению строятся конечные автоматы, представленные в виде разреженных булевых матриц, а затем ищется их пересечение с помощью тензорного произведения. Решением является транзитивное замыкание пересечения автоматов.

### 2. Достижимость для всего множества заданных вершин
По графу и регулярному выражению строятся конечные автоматы, представленные в виде разреженных булевых матриц, а затем выполняется синхронный поиск в ширину с помощью матричного произведения.

### 3. Достижимость для каждой вершины из заданного множества стартовых вершин
По графу и регулярному выражению строятся конечные автоматы, представленные в виде разреженных булевых матриц, а затем выполняется синхронный поиск в ширину с помощью матричного произведения. В отличии от задачи достижимости для всего множества заданных вершин, матрица, хранящая данные о нынешних состояних автоматов на текущей итерации поиска в ширину, имеет расширенный вид, ибо поддерживает информацию о каждой стартовой вершине отдельно.


## Цель эксперимента
Целью эксперимента являются ответы на следующие вопросы:
* Какое представление разреженных матриц и векторов лучше подходит для каждой из решаемых задач?
* Начиная с какого размера стартового множества выгоднее решать задачу для всех пар и выбирать нужные?
* На сколько решение второй задачи медленнее решения третьей при одинаковых начальных условиях?

## Описание набора данных для экспериментов
### Используемые графы
Эксперимент проводился на графах из датасета [CPFQ-data](https://jetbrains-research.github.io/CFPQ_Data/dataset/index.html): `skos`, `funding`, `gzip`, `bzip`. При выборе графов учитывался размер их алфваита меток. Оособо крупные графы не попали в перечень ввиду ограничений среды проведения эксперимента.

## Условия эксперимента ##

Данный раздел посвящен описанию характеристик оборудования, на котором производителись замеры, а также описание датасета.

### Характеристики оборудования ###

## Эксперимент

### Методика проведения замеров

Для проведения замеров времени выполнения регулярных запросов была написана функция `time`, которая позволяет заданное количество раз (в тестах автора это 100 раз) выполнить некоторый фрагмент кода, а также посчитать среднее время выполнения данного заданного фрагмента кода, а также узнать среднее квадратическое отклонение этого времени.

In [1]:
import random
import statistics

import prettytable
from prettytable import PrettyTable

from project.adjacency_matrix_csr import AdjacencyMatrixCsr
from project.adjacency_matrix_csc import AdjacencyMatrixCsc
from timeit import timeit

from networkx import MultiDiGraph
from pyformlang.regular_expression import Regex

from project.graph_utils import get_graph_info
from project.rpq_csr import rpq_tensor_csr, rpq_bfs_csr
from project.rpq_csc import rpq_tensor_csc, rpq_bfs_csc

import numpy as np

import pandas as pd

import cfpq_data


def generate_regexes(graph: MultiDiGraph) -> list:

    graph_data = get_graph_info(graph)

    def regex1():
        return "|".join(graph_data.edge_labels)

    def regex2():
        return random.choice(list(graph_data.edge_labels))

    def regex3():
        list_of_labels = list(graph_data.edge_labels)
        a, b = random.choices(list_of_labels, k=2)
        while b == a:
            b = random.choice(list_of_labels)
        return f'{a}.{a}*.{b}'

    return [regex1(), regex2(), regex3()]


test_graph_names = ["skos", "bzip", "funding", "gzip"]
test_graphs = list(map(lambda name: cfpq_data.graph_from_csv(cfpq_data.dataset.download(name)), test_graph_names))
test_regexes = list(map(lambda graph: generate_regexes(graph), test_graphs))

execution_times_number = 100


def time(stmt) -> tuple:

    times = []
    for i in range(execution_times_number):
        times.append(timeit(stmt=stmt, number=1))
    return statistics.mean(times), statistics.stdev(times)


def experiment_pass_rpq_tensor_csr(regex: str, graph: MultiDiGraph, start_nodes: list = None,
                                       final_nodes: list = None):

    rpq_tensor_csr(
        graph,
        regex,
        start_nodes,
        final_nodes
    )


def experiment_pass_rpq_bfs_csr(regex: str, graph: MultiDiGraph, all_reachable: bool,
                                                 start_nodes: list, final_nodes: list = None):
    rpq_bfs_csr(
        regex,
        graph,
        start_nodes,
        final_nodes,
        all_reachable
    )

def experiment_pass_rpq_tensor_csc(regex: str, graph: MultiDiGraph, start_nodes: list = None,
                                       final_nodes: list = None):

    rpq_tensor_csc(
        graph,
        regex,
        start_nodes,
        final_nodes
    )


def experiment_pass_rpq_bfs_csc(regex: str, graph: MultiDiGraph, all_reachable: bool,
                                                 start_nodes: list, final_nodes: list = None):
    rpq_bfs_csc(
        regex,
        graph,
        start_nodes,
        final_nodes,
        all_reachable
    )


def print_table(header: list, rows: list):

    table = PrettyTable()
    table.set_style(prettytable.MARKDOWN)
    table.field_names = header
    for row in rows:
        table.add_row(row)
    table.max_width = 50
    print(table)



[2023-01-26 06:18:45]>INFO>Found graph with name='skos'
[2023-01-26 06:18:45]>INFO>Load archive graph_archive=PosixPath('/home/z/.local/lib/python3.8/site-packages/cfpq_data/data/graphs/skos.tar.gz')
[2023-01-26 06:18:45]>INFO>Unzip graph name='skos' to file graph=PosixPath('/home/z/.local/lib/python3.8/site-packages/cfpq_data/data/graphs/skos/skos.csv')
[2023-01-26 06:18:45]>INFO>Remove archive graph_archive=PosixPath('/home/z/.local/lib/python3.8/site-packages/cfpq_data/data/graphs/skos.tar.gz')
[2023-01-26 06:18:45]>INFO>Load graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b6cef70> from path=PosixPath('/home/z/.local/lib/python3.8/site-packages/cfpq_data/data/graphs/skos/skos.csv')
[2023-01-26 06:18:45]>INFO>Found graph with name='bzip'
[2023-01-26 06:18:45]>INFO>Load archive graph_archive=PosixPath('/home/z/.local/lib/python3.8/site-packages/cfpq_data/data/graphs/bzip.tar.gz')
[2023-01-26 06:18:45]>INFO>Unzip graph name='bzip' to file graph=PosixPath('/home/z/.lo

### Какое представление разреженных матриц лучше подходит для каждой реализации алгоритма?
В обоих алгоритмах приходится умножать матрицы, и в обоих алгоритмах это основная часть. В связи с этим было решено провести эксперимент с выбором различных представлений матрицы на этапе умножения. Внутри scipy есть два вида матриц для высокопроизводительных операций:
* csr_matrix
* csc_matrix

Так как вид регулярного выражения влияет на длительность выполнения функции и количество умножений матриц внутри каждой из реализаций, выберем такое, которое дает наибольшее количество матричных умножений (регулярное выражение, допускающее любое слово, составленное из меток графа). Составим запросы к графам и проведем замеры:

In [2]:
start_nodes = list(map(lambda graph: cfpq_data.generate_multiple_source(graph, 1), test_graphs))

# csr test block
rpq_tensor = []
rpq_bfs = []
for i in range(len(test_graphs)):
    graph = test_graphs[i]
    regex = test_regexes[i][0]
    rpq_tensor.append((test_graph_names[i], '%.3f +- %.3f' % time(lambda: experiment_pass_rpq_tensor_csr(regex, graph, start_nodes=start_nodes[i])), '%.3f +- %.3f' % time(lambda: experiment_pass_rpq_tensor_csc(regex, graph, start_nodes=start_nodes[i]))))

# csc test block
regular_path_query_csc_results = []
bfs_based_regular_path_query_csc_results = []
for i in range(len(test_graphs)):
    graph = test_graphs[i]
    regex = test_regexes[i][0]
    rpq_bfs.append((test_graph_names[i], '%.3f +- %.3f' % time(lambda: experiment_pass_rpq_bfs_csr(regex, graph, all_reachable=False, start_nodes=start_nodes[i])), '%.3f +- %.3f' % time(lambda: experiment_pass_rpq_bfs_csc(regex, graph, all_reachable=False, start_nodes=start_nodes[i]))))

[2023-01-26 06:18:49]>INFO>Generate set of source vertices of 1 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b6cef70> for multiple-source evaluation
[2023-01-26 06:18:49]>INFO>Generate set of source vertices of 1 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b6cefa0> for multiple-source evaluation
[2023-01-26 06:18:49]>INFO>Generate set of source vertices of 1 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b6dbaf0> for multiple-source evaluation
[2023-01-26 06:18:49]>INFO>Generate set of source vertices of 1 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b4de040> for multiple-source evaluation
/home/z/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
/home/z/.local/lib/python3.8/site-packages/scipy

Результаты для `rpq_tensor`:

In [3]:

pd.DataFrame(
  rpq_tensor,
  columns=["name", "csr time", "csc time"],
)

,name,csr time,csc time
0,skos,0.018 +- 0.001,0.017 +- 0.001
1,bzip,0.016 +- 0.001,0.016 +- 0.001
2,funding,0.039 +- 0.001,0.037 +- 0.001
3,gzip,0.066 +- 0.003,0.065 +- 0.003


Результаты для `rpq_bfs`:

In [4]:
pd.DataFrame(
  rpq_bfs,
  columns=["name", "csr time", "csc time"],
)

,name,csr time,csc time
0,skos,0.031 +- 0.001,0.032 +- 0.001
1,bzip,0.016 +- 0.001,0.016 +- 0.001
2,funding,0.039 +- 0.001,0.039 +- 0.001
3,gzip,0.062 +- 0.002,0.062 +- 0.002


![](res/1_ready.png) 

Из полученных данных можно видеть, что на выбранных графах в алгоритмах нет существенной разницы, какой вид матриц применять для умножения. Различие в среднем времени обусловлено погрешностью.

### Начиная с какого размера начального множества задачу нахождения всех пар достижимых вершин выгоднее решать используя 1 алгоритм, чем 2 алгоритм?
Для ответа на поставленный вопрос проведем замеры, где будем брать различное число начальных вершин в каждом графе (будем брать от 5 % вершин, до 95 % вершин графа для теста). Также замеры будут проводиться при фиксированном представлении матрицы - csr.

In [7]:
size_of_start_nodes_in_percent = [i / 100 for i in range(5, 40, 15)]

result = []
for percent in size_of_start_nodes_in_percent:
    for i in range(len(test_graphs)):
        graph = test_graphs[i]
        for regex_num, value in enumerate(test_regexes[i]):
            start_nodes = list(cfpq_data.generate_multiple_source(graph, int(len(graph.nodes) * percent)))
            res_tensor = time(lambda: experiment_pass_rpq_tensor_csr(value, graph, start_nodes=start_nodes))
            res_bfs = time(lambda: experiment_pass_rpq_bfs_csr(value, graph, all_reachable=True, start_nodes=start_nodes))
            result.append((
                test_graph_names[i], 
                ".*" if regex_num == 0 else value, 
                percent, 
                '%.3f +- %.3f' % res_tensor, 
                '%.3f +- %.3f' % res_bfs,
                '%.3f' % (res_tensor[0] - res_bfs[0])
            ))

[2023-01-26 06:23:29]>INFO>Generate set of source vertices of 7 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b6cef70> for multiple-source evaluation
/home/z/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
[2023-01-26 06:23:42]>INFO>Generate set of source vertices of 7 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b6cef70> for multiple-source evaluation
[2023-01-26 06:23:44]>INFO>Generate set of source vertices of 7 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b6cef70> for multiple-source evaluation
[2023-01-26 06:23:47]>INFO>Generate set of source vertices of 31 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b6cefa0> for multiple-source evaluation
[2023-01-26 06:23:57]>INFO>Generate set of sour

Результаты:

In [8]:
pd.DataFrame(
  result,
  columns=["name", "regex", "vertex fraction", "rpq_tensor", "rpq_bfs", "diff"],
)

,name,regex,vertex fraction,rpq_tensor,rpq_bfs,diff
0,skos,.*,0.05,0.018 +- 0.001,0.108 +- 0.004,-0.089
1,skos,description,0.05,0.007 +- 0.001,0.012 +- 0.001,-0.005
2,skos,type.type*.example,0.05,0.007 +- 0.000,0.021 +- 0.001,-0.014
3,bzip,.*,0.05,0.016 +- 0.001,0.084 +- 0.002,-0.068
4,bzip,a,0.05,0.016 +- 0.001,0.052 +- 0.002,-0.037
5,bzip,d.d*.a,0.05,0.017 +- 0.001,0.106 +- 0.003,-0.090
6,funding,.*,0.05,0.038 +- 0.001,0.498 +- 0.010,-0.460
7,funding,range,0.05,0.027 +- 0.001,0.068 +- 0.002,-0.041
8,funding,creator.creator*.date,0.05,0.028 +- 0.001,0.074 +- 0.002,-0.046
9,gzip,.*,0.05,0.066 +- 0.003,0.511 +- 0.010,-0.445


Учитывая последний столбец diff можно сделать вывод, что в рамках выбранного датасета графов и регулярных выражений, реализаций автора, а также конфигурации системы, алоритм `rpq_bfs` показывает себя хуже во всех пробах. Это значит, что нужный размер начального множества, при котором алгоритм `rpq_tensor` показывает себя хуже второго, не был обнаружен в ходе эксперимента.

### На сколько реализация 2 алгоритм медленнее работает, если необходимо найти множества достижимых вершин для каждой из вершин начального множества?

Для ответа на поставленный вопрос проведем замеры, где будем брать различное число вершин в каждом графе (будем брать от 5 % вершин, до 95 % вершин для теста). Также замеры будут проводиться при фиксированном представлении матрицы - csr.

In [11]:
size_of_start_nodes_in_percent = [i / 100 for i in range(5, 40, 15)]

result = []
for percent in size_of_start_nodes_in_percent:
    for i in range(len(test_graphs)):
        graph = test_graphs[i]
        for regex_num, value in enumerate(test_regexes[i]):
            start_nodes = list(cfpq_data.generate_multiple_source(graph, int(len(graph.nodes) * percent)))
            result.append((
                test_graph_names[i], 
                ".*" if regex_num == 0 else value, 
                percent, 
                '%.3f +- %.3f' % time(lambda: experiment_pass_rpq_bfs_csr(value, graph, all_reachable=False, start_nodes=start_nodes)), 
                '%.3f +- %.3f' % time(lambda: experiment_pass_rpq_bfs_csr(value, graph, all_reachable=True, start_nodes=start_nodes)),
            ))

[2023-01-26 09:57:57]>INFO>Generate set of source vertices of 7 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b6cef70> for multiple-source evaluation
/home/z/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
[2023-01-26 09:58:10]>INFO>Generate set of source vertices of 7 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b6cef70> for multiple-source evaluation
[2023-01-26 09:58:12]>INFO>Generate set of source vertices of 7 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b6cef70> for multiple-source evaluation
[2023-01-26 09:58:15]>INFO>Generate set of source vertices of 31 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7ff49b6cefa0> for multiple-source evaluation
[2023-01-26 09:58:26]>INFO>Generate set of sour

In [13]:
pd.DataFrame(
  result,
  columns=["name", "regex", "vertex fraction", "all_reachable=False", "all_reachable=True"],
)

,name,regex,vertex fraction,all_reachable=False,all_reachable=True
0,skos,.*,0.05,0.033 +- 0.003,0.104 +- 0.003
1,skos,description,0.05,0.007 +- 0.000,0.012 +- 0.001
2,skos,type.type*.example,0.05,0.008 +- 0.000,0.016 +- 0.001
3,bzip,.*,0.05,0.029 +- 0.002,0.081 +- 0.003
4,bzip,a,0.05,0.020 +- 0.001,0.052 +- 0.002
5,bzip,d.d*.a,0.05,0.022 +- 0.001,0.107 +- 0.006
6,funding,.*,0.05,0.086 +- 0.005,0.509 +- 0.009
7,funding,range,0.05,0.028 +- 0.002,0.056 +- 0.002
8,funding,creator.creator*.date,0.05,0.029 +- 0.003,0.074 +- 0.003
9,gzip,.*,0.05,0.261 +- 0.006,0.497 +- 0.010


![](res/2_skos_1.png) ![](res/2_skos_2.png) ![](res/2_skos_3.png)
![](res/2_bzip_1.png) ![](res/2_bzip_2.png) ![](res/2_bzip_3.png)
![](res/2_funding_1.png) ![](res/2_funding_2.png) ![](res/2_funding_3.png)
![](res/2_gzip_1.png) ![](res/2_gzip_2.png) ![](res/2_gzip_3.png)

Из полученных данных можно видеть, что на выбранных графах реализация алгоритма `rpq_bfs` в режиме нахождения достижимых вершин из множества начальных быстрее, чем в режиме нахождения множества достижимых для каждой из множества начальных. Это ожидаемый результат, так как в "совместном" режиме на каждой итерации алгоритма выполняется намного меньше матричных умножений, а также другие матричные операции работают быстрее в силу меньшего размера самих используемых матриц.

# Заключение

Таким образом, благодаря поставленным экспериментам можно сделать вывод:
1. Какое представление разреженных матриц лучше подходит для каждой реализации алгоритма?
    Ответ: на предложенных реализациях алгоритмов разница между различными представлениями разреженных матриц несущественна.
2. Начиная с какого размера начального множества задачу нахождения всех пар достижимых вершин выгоднее решать используя 1 алгоритм, чем 2 алгоритм?
    Ответ: в среднем на любом размере начального множества предложенная реализация первого алгоритма быстрее, чем предложенная реализация 2-ого алгоритма.
3. На сколько реализация 2 алгоритм медленнее работает, если необходимо найти множества достижимых вершин для каждой из вершин начального множества?
    Ответ: реализация 2 алгоритма работает существенно медленнее, если необходимо найти множества достижимых вершин для каждой из вершин начального множества.